In [28]:
import pandas as pd
import json
import copy
import uuid

In [35]:
source_file_statements = "source_data/DDB Sellan Coding source.xlsx"
source_file_actants = "source_data/DDB Mockup ACTANTS (entities Person, Concepts, Locations, Objects from Sellan).xlsx"

In [23]:
with open('statement-json-structure-example.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data)

{'id': 1, 'text': 'David, strong friend of Peter, saw Adam in Brno. ', 'node_tree': {'id': '101', '_label': 'A: saw (A090)', 'type': 'root', 'actants': [{'id': 'P201', '_label': 'P: David (P201)', 'type': 'subject', 'props': [{'id': '501', '_label': 'A:has property', 'type': 'property', 'actants': [{'id': 'C601', '_label': 'C: friend_of (C601)', 'type': 'actant1', 'props': [{'id': '501', '_label': 'A:has property', 'type': 'property', 'actants': [{'id': 'C801', '_label': 'C: intensity (C801)', 'type': 'actant1', 'props': []}, {'id': 'C901', '_label': 'C: strong (C901)', 'type': 'actant2', 'props': []}]}]}, {'id': 'P701', '_label': 'P: Peter (P701)', 'type': 'actant2', 'props': []}], 'props': []}]}, {'id': 'P301', '_label': 'P: Adam (P301)', 'type': 'actant1', 'props': []}], 'props': [{'id': '401', '_label': 'A: has property', 'type': 'property', 'actants': [{'id': 'C201', '_label': 'C: location_where (C201)', 'type': 'actant1', 'props': []}, {'id': 'L301', '_label': 'L: Brno (L301)', '

In [63]:
statement_template = {
    "uuid": "",
    "source_id": "", # e.g. T3-1-01-013 if applicable
    "action_id": "", # A0090 (saw)
    "certainty": "",
    "epistemic_level": "",
    "meta": "",
    "modality": "",
    "node_tree": {},
    "note": "",
    "resources": [],
    "tags": [],
    "territories": [],
    "text": "David, strong friend of Peter, saw Adam in Brno. "
}

node_tree_root_template = {
    "uuid": "",
     "_label": "",  
    "type": "root",
    "actants": [],
    "props": []   
}

has_property_template = {  # A:has property
     "uuid": "",
     "_label": "A: has property",
    "type": "property",
    "actants": [],
    "props": []  
}

actant_template = {  # entity
     "uuid": "",
     "_label": "", #P Person
    "type": "",  # subject,  actant1, actant2
    "props": []  
}


In [5]:
# Pretty Printing JSON string back
print(json.dumps(data, indent = 4, sort_keys=True))

{
    "action_id": "A0090",
    "certainty": "",
    "epistemic_level": "",
    "id": 1,
    "meta": "",
    "modality": "",
    "node_tree": {
        "_label": "A: saw (A090)",
        "actants": [
            {
                "_label": "P: David (P201)",
                "id": "P201",
                "props": [
                    {
                        "_label": "A:has property",
                        "actants": [
                            {
                                "_label": "C: friend_of (C601)",
                                "id": "C601",
                                "props": [
                                    {
                                        "_label": "A:has property",
                                        "actants": [
                                            {
                                                "_label": "C: intensity (C801)",
                                                "id": "C801",
                                         

In [75]:
df_s = pd.read_excel(source_file_statements)
df_a = pd.read_excel(source_file_actants)


df_s = df_s.astype(str) 

In [77]:
#generate uuid for actants
def gen_uuid(row):
    return str(uuid.uuid4())
    
df_a['uuid'] = df_a.apply(lambda row: gen_uuid(row),axis=1) 

In [153]:
def create_object(template_dict):
    o = copy.deepcopy(template_dict)
    o['uuid'] = str(uuid.uuid4())
    return o

def create_statement(source_id=''):
    o = copy.deepcopy(statement_template)
    o['source_id'] = source_id
    o['uuid'] = str(uuid.uuid4())
    return o



def create_actant(atype = 'subject', source_id = ''):
    o = copy.deepcopy(actant_template)
    #search for actant in master dataframe  df_a
    search =  df_a.loc[df_a['codeid'] == source_id]
    uuid = search['uuid']

    if uuid.empty:
        return False
    
    o['type'] = atype
    o['uuid'] = str(uuid.item())
    
    return o

In [154]:
s = create_statement('source_id')
print(s)

a = create_actant('subject','P0004')
print(a)

{'uuid': '5a504f52-d7a8-41ea-bedc-b19a9da58199', 'source_id': 'source_id', 'action_id': '', 'certainty': '', 'epistemic_level': '', 'meta': '', 'modality': '', 'node_tree': {}, 'note': '', 'resources': [], 'tags': [], 'territories': [], 'text': 'David, strong friend of Peter, saw Adam in Brno. '}
{'uuid': '328b908e-9407-4bb0-bd43-9da35f35d923', '_label': '', 'type': 'subject', 'props': []}


In [164]:
statements = []  # list objectu statments

#idiot simple structure which ignores almost any complexity
#  1) does not work with multiples
#  2) needs C or P in id_subject


##############################################################
i = 0
for key, row in df_s.iterrows():
    i += 1
    if (i>5):
        #break
        pass
    
    #basic inteligence
    if ('P' not in row['id_subject'] and 'C' not in row['id_subject']):
        print ('Skipping '+  row['id'] + " : strange id_subject "+row['id_subject'])
        continue
    
    
    #cretate statement object
    s = create_statement(statement_template)
    s['source_id'] = row['id']
    s['action_id'] = row['id_action_or_relation']
    s['epistemic_level'] = row['epistemological_level']
    s['modality'] = row['modality']
    s['text'] = row['text']
    
    #create root action object
    ra = create_object(node_tree_root_template)
    #print(ra['uuid'])
    
    #create subject
    su = create_actant('subject',row['id_subject'])
    if su:
        su['_label'] = row['id_subject']
        ra['actants'].append(su)
    else:
        print('Subject '+row['id_subject']+" not recognized.")
    
    
    #create actant1
    if (row['id_actant1']):    
        a1 = create_actant('actant1',row['id_actant1'])
        if a1:
            a1['_label'] = row['id_actant1']
            ra['actants'].append(a1)
    else:
        print('Empty a1')
        pass
    
    #create actant2
    if (row['id_actant2']):    
        a2 = create_actant('actant2',row['id_actant2'])
        if (a2):
            a2['_label'] = row['id_actant2']
            ra['actants'].append(a2)
    else:
        print('Empty a2')
        pass
        
    s['node_tree'] = ra
    statements.append(s)

Skipping T3-1-00-001 : strange id_subject T3-1
Skipping T3-1-00-002 : strange id_subject T3-1
Subject P0356 #P0770 not recognized.
Skipping T3-1-00-003 : strange id_subject T3-1
Skipping T3-1-00-004 : strange id_subject T3-1
Skipping T3-1-00-005 : strange id_subject T3-1
Skipping T3-1-9918-010 : strange id_subject T3-1-9918-008
Skipping T3-1-9918-011 : strange id_subject T3-1-9918-009
Skipping T3-1-9919-009 : strange id_subject T3-1-9919-006
Skipping T3-1-9919-010 : strange id_subject T3-1-9919-007
Skipping T3-1-9919-011 : strange id_subject T3-1-9919-008
Skipping T3-1-9920-007 : strange id_subject T3-1-9920-003
Skipping T3-1-9920-008 : strange id_subject T3-1-9920-004
Skipping T3-1-9920-009 : strange id_subject T3-1-9920-005
Skipping T3-1-9921-016 : strange id_subject T3-1-9921-007
Skipping T3-1-9921-017 : strange id_subject T3-1-9921-008
Skipping T3-1-9921-018 : strange id_subject T3-1-9921-009
Skipping T3-1-9921-019 : strange id_subject T3-1-9921-010
Skipping T3-1-9921-020 : strange

Skipping T3-1-9969-014 : strange id_subject T3-1-9969-012
Skipping T3-1-9969-027 : strange id_subject T3-1-9969-018
Skipping T3-1-9969-028 : strange id_subject T3-1-9969-019
Skipping T3-1-9969-029 : strange id_subject T3-1-9969-020
Skipping T3-1-9969-030 : strange id_subject T3-1-9969-021
Skipping T3-1-9969-031 : strange id_subject T3-1-9969-022
Skipping T3-1-9969-032 : strange id_subject T3-1-9969-023
Skipping T3-1-9969-033 : strange id_subject T3-1-9969-024
Skipping T3-1-9969-034 : strange id_subject T3-1-9969-025
Skipping T3-1-9969-035 : strange id_subject T3-1-9969-026


In [156]:
statements

[{'uuid': '7635e5c0-88b6-4831-a6ce-d718411a9c43',
  'source_id': 'T3-1-01-001',
  'action_id': 'A0164',
  'certainty': '',
  'epistemic_level': '2-interpretive',
  'meta': '',
  'modality': 'nan',
  'node_tree': {'uuid': '95308bf4-09d6-451a-a570-998d54e455e6',
   '_label': '',
   'type': 'root',
   'actants': [{'uuid': '926b9e68-95d3-43cf-b02d-cdaf68b500a0',
     '_label': 'P0454',
     'type': 'subject',
     'props': []}],
   'props': []},
  'note': '',
  'resources': [],
  'tags': [],
  'territories': [],
  'text': 'nan'},
 {'uuid': '0aea2285-7fc7-4de2-87bf-8b391f7b259d',
  'source_id': 'T3-1-01-002',
  'action_id': 'A0093',
  'certainty': '',
  'epistemic_level': '2-interpretive',
  'meta': '',
  'modality': 'nan',
  'node_tree': {'uuid': 'a1e61e91-fc19-4d2c-8e6d-189fdac56a6e',
   '_label': '',
   'type': 'root',
   'actants': [{'uuid': '926b9e68-95d3-43cf-b02d-cdaf68b500a0',
     '_label': 'P0454',
     'type': 'subject',
     'props': []},
    {'uuid': '542beb73-eed8-4d4d-8d66-c9

In [166]:
df_a.to_excel('actants.xlsx')
df_a.to_csv('actants.csv')


statements_json = json.dumps(statements)
print(statements_json)

with open('statements.json', 'w') as file:
    file.write(statements_json)

[{"uuid": "7e7baa28-f476-456f-b9fe-5eba1df54578", "source_id": "T3-1-01-001", "action_id": "A0164", "certainty": "", "epistemic_level": "2-interpretive", "meta": "", "modality": "nan", "node_tree": {"uuid": "bdcedd61-0341-4e66-9978-58d276a309b8", "_label": "", "type": "root", "actants": [{"uuid": "926b9e68-95d3-43cf-b02d-cdaf68b500a0", "_label": "P0454", "type": "subject", "props": []}], "props": []}, "note": "", "resources": [], "tags": [], "territories": [], "text": "nan"}, {"uuid": "3ebbe271-fc43-4ba9-a00c-7a65d66601f7", "source_id": "T3-1-01-002", "action_id": "A0093", "certainty": "", "epistemic_level": "2-interpretive", "meta": "", "modality": "nan", "node_tree": {"uuid": "380e97ae-b127-4632-b8b4-1f4e96cecb08", "_label": "", "type": "root", "actants": [{"uuid": "926b9e68-95d3-43cf-b02d-cdaf68b500a0", "_label": "P0454", "type": "subject", "props": []}, {"uuid": "542beb73-eed8-4d4d-8d66-c969ad4392fc", "_label": "L0022", "type": "actant1", "props": []}], "props": []}, "note": "", "r